# 습도

POI별 관광객을 예측하기 위한 피쳐로써 날씨 데이터를 활용 
이 코드에서는 습도의 전처리를 진행함

[데이터셋 변수 설명] 
- format:day = 일자 데이터 (e.g. 1 = 01/01)
- hour = 시간(2자리) + 분(2자리) (e.g. 200 = 2:00 AM), UTC 기준
- forecast = hour를 기준으로 하여 몇 시간 뒤를 예측했는지
- value = 해당 시간에 대해 예측된 습도

[전처리 과정 설명] 
2:00(UTC시간대 기준)의 7시간 뒤의 예측값만을 추출하여 데이터프레임으로 구성하고자 함 
- hour == 200, forecast == 7인 행만을 조건 아래 추출 
- 누락 데이터 유무 확인
- 조건 색인 이후 불필요한 열인 hour, forecast를 drop

[기타 데이터 관련 설명] 
- 제주시 및 서귀포시 기준 행정동은 네이버 날씨의 대표값을 따름 (천지동의 경우 네이버 날씨에서는 서귀동을 기준으로 하지만 냘씨 예보는 법정동을 기준으로 하여 그에 준응하는 천지동의 데이터를 사용함) 
- 각 일자에 대한 예측값은 UTC기준 2:00AM (한국 시간, 11:00AM)에 발표된 해당 날짜의 18:00(KST)에 대한 예측값을 사용함

source : 기상청 - 기상자료개방포털 (2021/01-2021/06은 (구)단기 예보 파일셋 활용, 2021/07-2021/12은 단기 예보 파일셋 활용)
https://data.kma.go.kr/data/rmt/rmtList.do?code=420&pgmNo=572

In [7]:
import pandas as pd
from glob import glob

# 제주시 아라동

In [8]:
# glob()를 이용해 상반기, 하반기 습도 csv를 하나의 제주시 습도 데이터프레임으로 불러오기
file_names = glob("../Dataset/raw_data/weather/humidity/humidity_j/*.csv")
humidity_1 = pd.DataFrame() 


for file_name in file_names:
    temp = pd.read_csv(file_name, encoding='cp949')
    # 데이터프레임으로 불러오면서 칼럼명 재정의
    temp.rename(columns = {temp.columns[0]:'day', temp.columns[3]:'humidity(%)'}, inplace=True)

    humidity_1 = pd.concat([humidity_1, temp])


humidity_1



,day,hour,forecast,humidity(%)
0,1,200.0,4.0,65.0
1,1,200.0,7.0,75.0
2,1,200.0,10.0,75.0
3,1,200.0,13.0,75.0
4,1,200.0,16.0,75.0
...,...,...,...,...
93650,31,2300.0,60.0,85.0
93651,31,2300.0,61.0,85.0
93652,31,2300.0,62.0,85.0
93653,31,2300.0,63.0,85.0


In [9]:
# 정보 확인
humidity_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120448 entries, 0 to 93654
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   day          120448 non-null  object 
 1   hour         120260 non-null  float64
 2   forecast     120260 non-null  float64
 3   humidity(%)  120260 non-null  float64
dtypes: float64(3), object(1)
memory usage: 4.6+ MB


In [10]:
# 결측치 확인 
humidity_1.isnull().sum()

day              0
hour           188
forecast       188
humidity(%)    188
dtype: int64

In [11]:
# 조건 색인으로 데이터프레임 재구성
humidity_j = humidity_1[(humidity_1['hour'] == 200) & (humidity_1['forecast'] == 7)]

humidity_j


,day,hour,forecast,humidity(%)
1,1,200.0,7.0,75.0
149,2,200.0,7.0,75.0
297,3,200.0,7.0,75.0
445,4,200.0,7.0,75.0
593,5,200.0,7.0,75.0
...,...,...,...,...
91112,27,200.0,7.0,80.0
91621,28,200.0,7.0,85.0
92130,29,200.0,7.0,75.0
92639,30,200.0,7.0,65.0


In [12]:
# 누락 데이터 확인
humidity_j['day'].value_counts()

 1     12
 2     12
 28    12
 27    12
 26    12
 25    12
 24    12
 23    12
 22    12
 21    12
 20    12
 19    12
 18    12
 17    12
 16    12
 15    12
 14    12
 13    12
 12    12
 11    12
 10    12
 9     12
 8     12
 7     12
 6     12
 5     12
 4     12
 3     12
 29    11
 30    11
 31     7
Name: day, dtype: int64

In [13]:
# 재구성한 데이터프레임 결측치 확인
humidity_j.isnull().sum()

day            0
hour           0
forecast       0
humidity(%)    0
dtype: int64

In [14]:
# 불필요한 열 제거 후 float -> int형으로 변경
humidity_j = humidity_j.drop(columns = {'day','hour','forecast'}).astype(int)

humidity_j 

,humidity(%)
1,75
149,75
297,75
445,75
593,75
...,...
91112,80
91621,85
92130,75
92639,65


# 서귀포시 천지동

In [15]:
# glob()를 이용해 상반기, 하반기 습도 csv를 하나의 서귀포시 습도 데이터프레임으로 불러오기
file_names = glob("../Dataset/raw_data/weather/humidity/humidity_s/*.csv")
humidity_2 = pd.DataFrame() 


for file_name in file_names:
    temp = pd.read_csv(file_name, encoding='cp949')
    # 데이터프레임으로 불러오면서 칼럼명 재정의
    temp.rename(columns = {temp.columns[0]:'day', temp.columns[3]:'humidity(%)'}, inplace=True)

    humidity_2 = pd.concat([humidity_2, temp])


humidity_2

,day,hour,forecast,humidity(%)
0,1,200.0,4.0,55.0
1,1,200.0,7.0,60.0
2,1,200.0,10.0,65.0
3,1,200.0,13.0,65.0
4,1,200.0,16.0,65.0
...,...,...,...,...
93650,31,2300.0,60.0,70.0
93651,31,2300.0,61.0,70.0
93652,31,2300.0,62.0,70.0
93653,31,2300.0,63.0,75.0


In [16]:
# 정보 확인
humidity_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120448 entries, 0 to 93654
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   day          120448 non-null  object 
 1   hour         120260 non-null  float64
 2   forecast     120260 non-null  float64
 3   humidity(%)  120260 non-null  float64
dtypes: float64(3), object(1)
memory usage: 4.6+ MB


In [17]:
# 결측치 확인 
humidity_2.isnull().sum()

day              0
hour           188
forecast       188
humidity(%)    188
dtype: int64

In [18]:
# 조건 색인으로 데이터프레임 재구성
humidity_s = humidity_2[(humidity_2['hour'] == 200) & (humidity_2['forecast'] == 7)]

humidity_s

,day,hour,forecast,humidity(%)
1,1,200.0,7.0,60.0
149,2,200.0,7.0,70.0
297,3,200.0,7.0,70.0
445,4,200.0,7.0,60.0
593,5,200.0,7.0,65.0
...,...,...,...,...
91112,27,200.0,7.0,65.0
91621,28,200.0,7.0,70.0
92130,29,200.0,7.0,60.0
92639,30,200.0,7.0,50.0


In [19]:
# 누락 데이터 확인
humidity_s['day'].value_counts()

 1     12
 2     12
 28    12
 27    12
 26    12
 25    12
 24    12
 23    12
 22    12
 21    12
 20    12
 19    12
 18    12
 17    12
 16    12
 15    12
 14    12
 13    12
 12    12
 11    12
 10    12
 9     12
 8     12
 7     12
 6     12
 5     12
 4     12
 3     12
 29    11
 30    11
 31     7
Name: day, dtype: int64

In [20]:
# 재구성한 데이터프레임 결측치 확인
humidity_s.isnull().sum()

day            0
hour           0
forecast       0
humidity(%)    0
dtype: int64

In [21]:
# 불필요한 열 제거 후 float -> int형으로 변경
humidity_s = humidity_s.drop(columns = {'day','hour','forecast'}).astype(int)


humidity_s

,humidity(%)
1,60
149,70
297,70
445,60
593,65
...,...
91112,65
91621,70
92130,60
92639,50


# 데이터 병합

In [155]:
# 날짜 데이터 불러오기
date = pd.read_csv('./data/date_data.csv')


In [156]:
# 제주시 습도 데이터와 날짜 데이터 병합
jeju = pd.concat([date.reset_index(drop=True), humidity_j.reset_index(drop=True)],axis=1)

jeju

,Unnamed: 0,날짜,공휴일,요일,humidity(%)
0,0,2021-01-01,1,금,75
1,1,2021-01-02,0,토,75
2,2,2021-01-03,0,일,75
3,3,2021-01-04,0,월,75
4,4,2021-01-05,0,화,75
...,...,...,...,...,...
360,360,2021-12-27,0,월,80
361,361,2021-12-28,0,화,85
362,362,2021-12-29,0,수,75
363,363,2021-12-30,0,목,65


In [157]:
# 불필요한 열 제거
jeju = jeju.drop(columns = 'Unnamed: 0')

jeju

,날짜,공휴일,요일,humidity(%)
0,2021-01-01,1,금,75
1,2021-01-02,0,토,75
2,2021-01-03,0,일,75
3,2021-01-04,0,월,75
4,2021-01-05,0,화,75
...,...,...,...,...
360,2021-12-27,0,월,80
361,2021-12-28,0,화,85
362,2021-12-29,0,수,75
363,2021-12-30,0,목,65


In [159]:
# 제주시 데이터 csv 저장
jeju.to_csv("C:/Users/PC/Desktop/MTWT/data/humidity_j/jeju.csv", index = False)

In [160]:
# 서귀포시 습도 데이터와 날짜 데이터 병합
seogwipo = pd.concat([date.reset_index(drop=True), humidity_s.reset_index(drop=True)],axis=1)

seogwipo

,Unnamed: 0,날짜,공휴일,요일,humidity(%)
0,0,2021-01-01,1,금,60
1,1,2021-01-02,0,토,70
2,2,2021-01-03,0,일,70
3,3,2021-01-04,0,월,60
4,4,2021-01-05,0,화,65
...,...,...,...,...,...
360,360,2021-12-27,0,월,65
361,361,2021-12-28,0,화,70
362,362,2021-12-29,0,수,60
363,363,2021-12-30,0,목,50


In [161]:
# 불필요한 열 제거
seogwipo = seogwipo.drop(columns = 'Unnamed: 0')

seogwipo

,날짜,공휴일,요일,humidity(%)
0,2021-01-01,1,금,60
1,2021-01-02,0,토,70
2,2021-01-03,0,일,70
3,2021-01-04,0,월,60
4,2021-01-05,0,화,65
...,...,...,...,...
360,2021-12-27,0,월,65
361,2021-12-28,0,화,70
362,2021-12-29,0,수,60
363,2021-12-30,0,목,50


In [162]:
# 서귀포시 데이터 csv 저장
seogwipo.to_csv("C:/Users/PC/Desktop/MTWT/data/humidity_s/seogwipo.csv", index = False)